In [1]:
### IMPORTS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU
# from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras import backend
from keras import optimizers
import keras
import folium
from folium.plugins import AntPath, MousePosition
from sklearn.utils import shuffle, resample
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Dropout, LSTM, Input, RepeatVector, GRU, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
from datetime import datetime
import random
import math
from tqdm import tqdm
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from sklearn.metrics import silhouette_score
from scipy.stats import gaussian_kde
import seaborn as sns
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon


In [2]:
### DATASET LOAD

df = pd.read_csv(r"D:\Python\master thesis\data\final_dataset.csv")
# df = pd.read_csv(r"D:\Python\master thesis\collision data\final_dataset_collision.csv")
df = df.drop(columns=['Unnamed: 0'])
df['# Timestamp'] = pd.to_datetime(df['# Timestamp'])
df = df.sort_values(by=['MMSI', '# Timestamp'])
# # # Display the first few rows of the dataset to understand its structure

In [27]:
### Differences of Lat and Long conversion to coordinates
def diff_to_coord(y,X):
    a = np.zeros_like(y)
    for j in tqdm(range(a.shape[0])):
        for i in range(a.shape[1]):
            if i == 0:
                a[j,i,0] = X[j,-1,0] - y[j,i,0]
                a[j,i,1] = X[j,-1,1] - y[j,i,1]
            else:
                a[j,i,0] = a[j,i-1,0] - y[j,i,0]
                a[j,i,1] = a[j,i-1,1] - y[j,i,1]
    return a

In [ ]:
### Sequencing
seq_input_length = 30
seq_output_length = 20
seq_length = seq_input_length+seq_output_length
window_size = seq_length // 2

seq = []
for i in tqdm(range(0, len(df) - seq_length + 1, window_size)):
    temp = df[i: i + seq_length].values
    # check if all sequences hold only one MMSI (first == last)
    if(temp[0,0] == temp[-1,0]):
        seq.append(temp)
seq = np.dstack(seq)
seq = np.rollaxis(seq,-1)
# seq = shuffle(seq,random_state=42)
seq.shape

In [ ]:
### Removal of bad sequences e.g. SOG = 0, irregular timesteps, calculated speed = 0, NaNs, illogical speed
bad_sequences = []
for index,i in tqdm(enumerate(seq)):
    # print(index)
    if 0 in i[:,4] or (i[:,6] > 2).sum() > 0 or (i[:,6] < 1).sum() > 0 or 0 in (np.append(np.atleast_2d(i[:,7:9].sum(axis=1)).T,i[:,7:8],axis=1)).sum(axis=1) or np.isnan((i[:,2:].astype(np.float32))).sum() > 0 or (i[:,11] <= 0).sum() > 0 or (i[:,11] > 771).sum() > 0:
        bad_sequences.append(index)
new_seq = np.delete(seq,bad_sequences,axis=0)
new = new_seq.reshape((new_seq.shape[0]*new_seq.shape[1],new_seq.shape[2]))

In [32]:
df = pd.DataFrame(new,columns=['MMSI','# Timestamp','Latitude', 'Longitude' ,'SOG','Heading','DateDiff','Lat_speed','Long_speed','Lat_lag','Long_lag','haversine_distance'])

In [33]:
### Renewed deltas
df['d_Lat'] =  df['Lat_lag'] - df['Latitude']
df['d_Long'] = df['Long_lag'] - df['Longitude']

In [34]:
### NORMALIZING VALUES
### if scaler file exists skip this part
small_df = df[['MMSI','# Timestamp']]
df = df.drop(columns=['MMSI','# Timestamp'])
features=[i for i in df.columns]
scaler = MinMaxScaler()
 
small_df[features] = scaler.fit_transform(df[features])
df = small_df

In [28]:
### Scaler file
import joblib
joblib.dump(scaler, 'sc.joblib') 
scaler = joblib.load('sc.joblib')

In [35]:
df_numpy = df.to_numpy()
seq = df_numpy.reshape((new_seq.shape[0],new_seq.shape[1],df.shape[1]))

In [36]:
### Dividing inot predictor (X) and target variables (y)
X = seq[:,0:seq_input_length,:]
y = seq[:,seq_input_length:seq_length,:]

In [38]:
### SPLITTING DATA INTO TRAIN,TEST,VALIDATION (70%,15%,15%)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.176,random_state=1)   # 70 : 15 : 15


X_train_primary = X_train
X_test_primary = X_test
X_val_primary = X_val
y_train_primary = y_train
y_test_primary = y_test
y_val_primary = y_val


X_train = X_train[:,:,2:]
X_test = X_test[:,:,2:]
X_val = X_val[:,:,2:]

# y_train = y_train[:,:,2:4] # drop all features except lat and long
# y_val = y_val[:,:,2:4] # drop all features except lat and long
# y_test = y_test[:,:,2:4] # drop all features except lat and long

y_train = y_train[:,:,-2:] # drop all features except lat and long
y_val = y_val[:,:,-2:] # drop all features except lat and long
y_test = y_test[:,:,-2:] # drop all features except lat and long

X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
X_val=X_val.astype('float32')
y_train=y_train.astype('float32')
y_test=y_test.astype('float32')
y_val=y_val.astype('float32')

y_train_flat = np.copy(y_train)
y_val_flat = np.copy(y_val)
y_test_flat = np.copy(y_test)
y_train_flat = y_train_flat.reshape(y_train_flat.shape[0], y_train_flat.shape[1]*y_train_flat.shape[2])
y_val_flat = y_val_flat.reshape(y_val_flat.shape[0], y_val_flat.shape[1]*y_val_flat.shape[2])
y_test_flat = y_test_flat.reshape(y_test_flat.shape[0], y_test_flat.shape[1]*y_test_flat.shape[2])

### Creating mock colissions

In [39]:
## Data prep before mock collisions

y_test_copy = np.concatenate((np.zeros((y_test.shape[0], y_test.shape[1], X_test.shape[2] - y_test.shape[2])),y_test),axis=2)
y_test_copy = scaler.inverse_transform(y_test_copy.reshape(y_test_copy.shape[0]*y_test_copy.shape[1],y_test_copy.shape[2]))[:,-2:]
y_test_copy = y_test_copy.reshape((int(y_test_copy.shape[0]/seq_output_length),seq_output_length,2))

X_test = scaler.inverse_transform(X_test.reshape(X_test.shape[0]*X_test.shape[1],X_test.shape[2]))
X_test = X_test.reshape((int(X_test.shape[0]/seq_input_length),seq_input_length,X_train.shape[2]))
y_test_copy = diff_to_coord(y_test_copy,X_test)

y_test_copy = np.concatenate((y_test_primary[:,:,:2],y_test_copy),axis=2)
X_test = np.concatenate((X_test_primary[:,:,:2],X_test),axis=2)

In [42]:
def mockup_generator(X_test,y_test,random_ship=True,ship_index1=None,ship_index2=None,timestamp_index1=None,timestamp_index2=None):
    """
    Returns: 
        random_index- index of first sequence
        ship_index - index of second sequence
        mmsi - mmsi of ship of first sequence
        y_test[ship_index,0,0] - mmsi of ship of first sequence
        new_traj_X - changed first sequence predictor variable to imitate collision
        new_traj_y - changed first sequence target variable to imitate collision
        timestamp_index - timestamp index of first sequence
        time_index - timestamp index of second sequence
    """
    if random_ship:
        random_index= random.randint(0,X_test.shape[0])
        timestamp_index=random.randint(y_test.shape[2]//4,y_test.shape[2]//4*3)
        timestamp=y_test[random_index,timestamp_index,1]
        mmsi=y_test[random_index,0,0]
        index_list = []
        for i in range(y_test.shape[0]):
            if timestamp in y_test[i,:,1] and mmsi != y_test[i,0,0]:
                index_list.append(i)
        if len(index_list) == 0:
            mockup_generator(X_test,y_test)
        else:
            ship_index = random.choice(index_list)
            time_index= np.where(y_test[ship_index,:,1]==timestamp)[0][0]
            differences = y_test[random_index,timestamp_index,2:4] - y_test[ship_index,time_index,2:4]
            colission_path_X = X_test[random_index,:,2:4] - differences
            colission_path_y = y_test[random_index,:,2:4] - differences
            
            plt.plot(X_test[random_index,:,3],X_test[random_index,:,2])
            plt.plot(y_test[random_index,:,3],y_test[random_index,:,2])
            plt.plot(X_test[ship_index,:,3],X_test[ship_index,:,2],label=f'start_path {y_test[ship_index,0,0]}',)
            plt.plot(y_test[ship_index,:,3],y_test[ship_index,:,2],label=f'end_path {y_test[ship_index,0,0]}')
            plt.plot(colission_path_X[:,1],colission_path_X[:,0],label=f'start_colission_path {y_test[random_index,0,0]}')
            plt.plot(colission_path_y[:,1],colission_path_y[:,0],label=f'end_colission_path {y_test[random_index,0,0]}')
            plt.legend()
            plt.show()

            plt.plot(X_test[ship_index,:,3],X_test[ship_index,:,2],label=f'start_path {y_test[ship_index,0,0]}',)
            plt.plot(y_test[ship_index,:,3],y_test[ship_index,:,2],label=f'end_path {y_test[ship_index,0,0]}')
            plt.plot(colission_path_X[:,1],colission_path_X[:,0],label=f'start_colission_path {y_test[random_index,0,0]}')
            plt.plot(colission_path_y[:,1],colission_path_y[:,0],label=f'end_colission_path {y_test[random_index,0,0]}')
            plt.legend()
            plt.show()
            # print(X_test.shape)
            # print(colission_path_X.shape)
            # print(X_test[random_index,:,:])
            # print(colission_path_X)

            X_to_change = X_test[random_index,:,:].copy()
            X_to_change[:,2:4] = colission_path_X
            X_to_change[:,9:11] = X_to_change[:,2:4] + X_to_change[:,12:14]
            y_to_change = y_test[random_index,:,:].copy()
            y_to_change[:,2:4] = colission_path_y
            new_traj_X =  X_to_change
            new_traj_y = y_to_change
            return random_index, ship_index ,mmsi ,y_test[ship_index,0,0],new_traj_X, new_traj_y,timestamp_index,time_index
    elif not random_ship:
        differences = y_test[ship_index1,timestamp_index1,2:4] - y_test[ship_index2,timestamp_index2,2:4]
        colission_path_X = X_test[ship_index1,:,2:4] - differences
        colission_path_y = y_test[ship_index1,:,2:4] - differences
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
        ax1.set_title(f"Original trajectories for vessels:\n$MMSI_1$ = {round(y_test[ship_index1,0,0])}\n$MMSI_2$ = {round(y_test[ship_index2,0,0])}")
        ax1.plot(X_test[ship_index1,:,3],X_test[ship_index1,:,2],label=f'Original predictor path for vessel MMSI = {y_test[ship_index1,0,0]}',color='#001219')
        ax1.plot(y_test[ship_index1,:,3],y_test[ship_index1,:,2],label=f'Original target path for vessel MMSI = {y_test[ship_index1,0,0]}',color='#0a9396')
        ax1.plot(X_test[ship_index2,:,3],X_test[ship_index2,:,2],label=f'Original predictor path for vessel MMSI = {y_test[ship_index2,0,0]}',color='#9b2226')
        ax1.plot(y_test[ship_index2,:,3],y_test[ship_index2,:,2],label=f'Original target path for vessel MMSI = {y_test[ship_index2,0,0]}',color='#ee9b00')
        ax1.set_xlabel('Longitude')
        ax1.set_ylabel('Latitude')
        # plt.plot(colission_path_X[:,1],colission_path_X[:,0],label=f'Artificial predictor path for vessel MMSI = {y_test[ship_index1,0,0]}')
        # plt.plot(colission_path_y[:,1],colission_path_y[:,0],label=f'Artificial target path for vessel MMSI = {y_test[ship_index1,0,0]}')
        # plt.legend(fontsize='x-small')
        # plt.show()
        ax2.set_title(f"Mock collision trajectories for vessels:\n$MMSI_1$ = {round(y_test[ship_index1,0,0])}\n$MMSI_2$ = {round(y_test[ship_index2,0,0])}")
        # plt.plot(X_test[ship_index1,:,3],X_test[ship_index1,:,2],label=f'Original predictor path for vessel MMSI = {y_test[ship_index1,0,0]}')
        # plt.plot(y_test[ship_index1,:,3],y_test[ship_index1,:,2],label=f'Original target path for vessel MMSI = {y_test[ship_index1,0,0]}')
        ax2.plot(X_test[ship_index2,:,3],X_test[ship_index2,:,2],label=f'Original predictor path for vessel MMSI = {y_test[ship_index2,0,0]}',color='#9b2226')
        ax2.plot(y_test[ship_index2,:,3],y_test[ship_index2,:,2],label=f'Original target path for vessel MMSI = {y_test[ship_index2,0,0]}',color='#ee9b00')
        ax2.plot(colission_path_X[:,1],colission_path_X[:,0],label=f'Artificial predictor path for vessel MMSI = {y_test[ship_index1,0,0]}',color='#001219')
        ax2.plot(colission_path_y[:,1],colission_path_y[:,0],label=f'Artificial target path for vessel MMSI = {y_test[ship_index1,0,0]}',color='#0a9396')
        ax2.set_xlabel('Longitude')
        # plt.legend(fontsize='x-small')
        plt.locator_params(axis='both', nbins=4) 
        plt.show()
        X_to_change = X_test[ship_index1,:,:].copy()
        X_to_change[:,2:4] = colission_path_X
        X_to_change[:,9:11] = X_to_change[:,2:4] + X_to_change[:,12:14]
        y_to_change = y_test[ship_index1,:,:].copy()
        y_to_change[:,2:4] = colission_path_y
        new_traj_X =  X_to_change
        new_traj_y = y_to_change
        return ship_index1, ship_index2 ,y_test[ship_index1,0,0] ,y_test[ship_index2,0,0],new_traj_X, new_traj_y,timestamp_index1,timestamp_index2

In [ ]:
###  RANDOM COLLISIONS
### Skip this if collisions are mock collisions are already chosen

X_changes = []
y_changes = []
mmsis = []
first_indexes = []
second_indexes = []
all_indexes = []
time_indexes_1 = []
time_indexes_2 = []
while len(X_changes)<10:
    first_idx,second_idx,first_mmsi,second_mmsi,new_traj_X,new_traj_y,first_timestamp_index,second_timestamp_index = mockup_generator(X_test,y_test_copy)
    ok = 'y'
    ok = input('y or n?')
    if first_idx in all_indexes or second_idx in all_indexes or ok != 'y':
        continue
    X_changes.append(new_traj_X)
    y_changes.append(new_traj_y)
    mmsis.append(first_mmsi)
    mmsis.append(second_mmsi)
    first_indexes.append(first_idx)
    second_indexes.append(second_idx)
    all_indexes.append(first_idx)
    all_indexes.append(second_idx)
    time_indexes_1.append(first_timestamp_index)
    time_indexes_2.append(second_timestamp_index)

for i,idx in enumerate(first_indexes):
    X_test[idx,:,:] = X_changes[i]
    y_test_copy[idx,:,:] = y_changes[i]

X_test = X_test[:,:,2:]
y_test_copy = y_test_copy[:,:,2:]

y_test_copy = np.concatenate((y_test_copy,np.zeros((y_test_copy.shape[0],y_test_copy.shape[1],scaler.n_features_in_ - y_test_copy.shape[2]))),axis=2)
y_test_copy = scaler.transform(y_test_copy.reshape(y_test_copy.shape[0]*y_test_copy.shape[1],y_test_copy.shape[2])).reshape((y_test_copy.shape[0],y_test_copy.shape[1],y_test_copy.shape[2]))
y_test_copy = y_test_copy[:,:,:2]


X_test = scaler.transform(X_test.reshape(X_test.shape[0]*X_test.shape[1],X_test.shape[2])).reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2]))

In [ ]:
# NOT RANDOM COLISSIONS
mmsis  = [304776000.0, 210393000.0, 304146000.0, 245271000.0, 265413000.0, 259030000.0, 477587000.0, 636021860.0, 314418000.0, 255815000.0, 246000000.0, 244130641.0, 311379000.0, 255805594.0, 255815000.0, 255989000.0, 314500000.0, 244140468.0, 538006410.0, 255805577.0]
first_indexes = [10149, 7141, 2348, 4980, 10218, 1372, 1149, 2351, 4576, 9205]
second_indexes = [9876, 4438, 5087, 2156, 10050, 2538, 532, 8929, 9491, 3898]
all_indexes  = [10149, 9876, 7141, 4438, 2348, 5087, 4980, 2156, 10218, 10050, 1372, 2538, 1149, 532, 2351, 8929, 4576, 9491, 9205, 3898]
time_indexes_1  = [2, 2, 3, 3, 2, 1, 3, 2, 2, 2]
time_indexes_2  = [4, 12, 4, 16, 16, 19, 17, 13, 14, 6]

X_changes = []
y_changes = []
for i in range(len(first_indexes)):
    print(i+1)
    first_idx,second_idx,first_mmsi,second_mmsi,new_traj_X,new_traj_y,first_timestamp_index,second_timestamp_index = mockup_generator(X_test,y_test_copy,random_ship = False,ship_index1=first_indexes[i],ship_index2=second_indexes[i],timestamp_index1=time_indexes_1[i],timestamp_index2=time_indexes_2[i])
    X_changes.append(new_traj_X)
    y_changes.append(new_traj_y)


for i,idx in enumerate(first_indexes):
    X_test[idx,:,:] = X_changes[i]
    y_test_copy[idx,:,:] = y_changes[i]

    
X_test = X_test[:,:,2:]
y_test_copy = y_test_copy[:,:,2:]

y_test_copy = np.concatenate((y_test_copy,np.zeros((y_test_copy.shape[0],y_test_copy.shape[1],scaler.n_features_in_ - y_test_copy.shape[2]))),axis=2)
y_test_copy = scaler.transform(y_test_copy.reshape(y_test_copy.shape[0]*y_test_copy.shape[1],y_test_copy.shape[2])).reshape((y_test_copy.shape[0],y_test_copy.shape[1],y_test_copy.shape[2]))
y_test_copy = y_test_copy[:,:,:2]


X_test = scaler.transform(X_test.reshape(X_test.shape[0]*X_test.shape[1],X_test.shape[2])).reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2]))
collisions = np.array([sorted(pair) for pair in zip(first_indexes, second_indexes)])
    

In [ ]:
print(f'mmsis  = {mmsis }')
print(f'first_indexes = {first_indexes}')
print(f'second_indexes = {second_indexes}')
print(f'all_indexes  = {all_indexes }')
print(f'time_indexes_1  = {time_indexes_1 }')
print(f'time_indexes_2  = {time_indexes_2 }')
# mmsis  = [304776000.0, 210393000.0, 304146000.0, 245271000.0, 265413000.0, 259030000.0, 477587000.0, 636021860.0, 314418000.0, 255815000.0, 246000000.0, 244130641.0, 311379000.0, 255805594.0, 255815000.0, 255989000.0, 314500000.0, 244140468.0, 538006410.0, 255805577.0]
# first_indexes = [10149, 7141, 2348, 4980, 10218, 1372, 1149, 2351, 4576, 9205]
# second_indexes = [9876, 4438, 5087, 2156, 10050, 2538, 532, 8929, 9491, 3898]
# all_indexes  = [10149, 9876, 7141, 4438, 2348, 5087, 4980, 2156, 10218, 10050, 1372, 2538, 1149, 532, 2351, 8929, 4576, 9491, 9205, 3898]
# time_indexes_1  = [2, 2, 3, 3, 2, 1, 3, 2, 2, 2]
# time_indexes_2  = [4, 12, 4, 16, 16, 19, 17, 13, 14, 6]

## Modelling

In [ ]:
### MODELLING

def create_and_compile_model(model_type, layer_size,num_epochs,batch_size,n_features,available_models,test_number):
    if model_type not in available_models:
        print('No such model available, available models: ',available_models)
        return 
    model_file_name = f'models/{model_type}-cells-{layer_size}-epochs-{num_epochs}-batch-{batch_size}-test-{test_number}-datetime-{datetime.today().strftime("%Y_%m_%d-%H_%M_%S")}.h5'
    history_file_name = f'models/{model_type}-cells-{layer_size}-epochs-{num_epochs}-batch-{batch_size}-{test_number}-datetime-{datetime.today().strftime("%Y_%m_%d-%H_%M_%S")}-history.npy'
    if model_type == 'GRU':
        model = Sequential()
        model.add(GRU(layer_size,activation='relu',input_shape=(seq_input_length,n_features)))
        model.add(Dropout(0.01))
        model.add(Dense(y_train_flat.shape[1],activation='relu'))
    elif model_type == 'Bi_LSTM':
        model = Sequential()
        model.add(Bidirectional(LSTM(layer_size, activation='relu'), input_shape=(seq_input_length, n_features)))
        model.add(Dropout(0.01))
        model.add(Dense(y_train_flat.shape[1], activation='relu'))
    elif model_type == 'LSTM':
        model = Sequential()
        model.add(LSTM(layer_size, activation='relu', input_shape=(seq_input_length, n_features)))
        model.add(Dropout(0.01))
        model.add(Dense(y_train_flat.shape[1], activation='relu'))
    elif model_type == 'LSTM_AE':
        model = Sequential()
        model.add(LSTM(layer_size, activation='relu', input_shape=(seq_input_length, n_features)))
        model.add(Dropout(0.01))
        model.add(RepeatVector(seq_output_length))
        model.add(LSTM(layer_size, activation='relu', return_sequences=False))
        model.add(Dropout(0.01))
        model.add(Dense(y_train_flat.shape[1], activation='relu'))

    reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, verbose=0,patience=8, min_lr=0.000001)
    checkpoint = ModelCheckpoint(model_file_name, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
    model.compile(optimizer='adam',loss='mse')
    history = model.fit(X_train, y_train_flat, epochs=num_epochs, batch_size=batch_size, verbose=1, validation_data=(X_val, y_val_flat), callbacks=[earlyStopping,reduce_lr,checkpoint])
    np.save(history_file_name, history.history)
    model.save(model_file_name)

In [ ]:
layer_sizes = [100,200]
num_epochs = 100
batch_sizes = [64,128]
n_features = X_train.shape[2]
model_repeat_time = 1
available_models = ['GRU','LSTM_AE','LSTM','Bi_LSTM']
# available_models = ['Bi_LSTM']
for test_number in range(model_repeat_time):
    for model in available_models:
        for layer_size in layer_sizes:
            for batch_size in batch_sizes:
                create_and_compile_model(model,layer_size,num_epochs,batch_size,n_features,available_models,test_number)

# truksta tik bilstm

# **Predictions start**

In [44]:
def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [ ]:
### Predict on y_test (10 different models)

y_test = np.concatenate((np.zeros((y_test.shape[0], y_test.shape[1], X_test.shape[2] - y_test.shape[2])),y_test),axis=2)
y_test = scaler.inverse_transform(y_test.reshape(y_test.shape[0]*y_test.shape[1],y_test.shape[2]))[:,-2:]
y_test = y_test.reshape((int(y_test.shape[0]/seq_output_length),seq_output_length,2))

model_names = []
predictions = []
for model_name in os.listdir('D:\Python\master thesis\master-thesis\models'):
    if '.h5' in model_name:
        print(model_name)
        model = load_model(f'D:\Python\master thesis\master-thesis\models\{model_name}')
        y_pred_test = model.predict(X_test)
        y_pred_test= y_pred_test.reshape(y_pred_test.shape[0],y_test.shape[1],y_test.shape[2])

        y_pred_test = np.concatenate((np.zeros((y_pred_test.shape[0], y_pred_test.shape[1], X_test.shape[2] - y_test.shape[2])),y_pred_test),axis=2)
        y_pred_test = scaler.inverse_transform(y_pred_test.reshape(y_pred_test.shape[0]*y_pred_test.shape[1],y_pred_test.shape[2]))[:,-2:]
        y_pred_test = y_pred_test.reshape((int(y_pred_test.shape[0]/seq_output_length),seq_output_length,2))
        
        model_names.append(model_name)
        predictions.append(y_pred_test)


In [ ]:
### Predictions data prep

X_test = scaler.inverse_transform(X_test.reshape(X_test.shape[0]*X_test.shape[1],X_test.shape[2]))
X_test = X_test.reshape((int(X_test.shape[0]/seq_input_length),seq_input_length,X_train.shape[2]))
y_test = diff_to_coord(y_test,X_test)
predictions_new = []
for y_pred in predictions:
    y_pred_test = diff_to_coord(y_pred,X_test)
    predictions_new.append(y_pred_test)
predictions = predictions_new

y_test_new = np.concatenate((y_test_primary[:,:,:2],y_test),axis=2)
X_test_new = np.concatenate((X_test_primary[:,:,:2],X_test),axis=2)
predictions_new = []
for i in predictions:
    y_pred_test_new = np.concatenate((y_test_primary[:,:,:2],i),axis=2)
    predictions_new.append(y_pred_test_new)
predictions = predictions_new

new_predictions= np.stack(predictions,axis=-1)
centroids = np.concatenate([new_predictions[:,:,0:2,0],np.mean(new_predictions[:,:,2:4,:],axis=-1)],axis=-1)

In [51]:
#if doesnt exist procceed with lower
interactions = np.load('interactions.npy')

In [ ]:
### interaction dataset prep, result - table with three columns : first sequence id, second sequence id, whether the interaction is collision.

total_interactions = 0
interactions = []
for i in tqdm(range(X_test_new.shape[0])):
    for j in range(X_test_new.shape[0]):
        if j > i and X_test_new[i,0,0] != X_test_new[j,0,0] and not(X_test_new[i,-1,1] + np.timedelta64(19,'m') < X_test_new[j,-1,1] or X_test_new[j,-1,1] + np.timedelta64(19,'m') < X_test_new[i,-1,1]) and haversine_distance(X_test_new[i,-1,3],X_test_new[i,-1,2],X_test_new[j,-1,3],X_test_new[j,-1,2]) < 30.84:
            total_interactions+=1
            if np.any(np.all(collisions == [i,j],axis=1)):
                interactions.append([i,j,1])
            else:
                interactions.append([i,j,0])
                
            
print(total_interactions)

In [ ]:
### Check

sample = random.randint(0,X_test.shape[0] - 1)
# sample = 1681
for y_pred in predictions:
    plt.plot(y_pred[sample,:,3],y_pred[sample,:,2])
    # break
plt.plot(X_test[sample,:,1],X_test[sample,:,0], color='red',label= 'start_path')
plt.plot(y_test[sample,:,1],y_test[sample,:,0], color='blue',label= 'true_path')
plt.legend()

# **Predictor**

In [52]:
def vizualize_ellipse(prediction_region,with_points=True,legend=True,label=True,vizualize=True): #Vizualizes bounding ellipse around a bi-variate cluster
    t = np.linspace(0,2.0 * np.pi,1000)
    xy = np.stack((np.cos(t),np.sin(t)),axis=-1)
    covariance = np.cov(prediction_region[1,:].astype('float'),prediction_region[0,:].astype('float'))
    eigenvalues, eigenvectors = np.linalg.eig(covariance)
    val = np.sqrt(eigenvalues)
    center = np.mean([prediction_region[1,:].astype('float'),prediction_region[0,:].astype('float')],axis=1)[:,None]
    if vizualize:
        if with_points:
            plt.scatter(prediction_region[1,:],prediction_region[0,:],label='Predicted points',color='#1f77bd')
            plt.scatter(center[0],center[1],label='Mean prediction',color='#ff7f0e')
        if label:
            plt.plot(*(2.44765*eigenvectors @ (val * xy).T + center),label='Predicted trajectory regions',color='#2ca02c')
        else:
            plt.plot(*(2.44765*eigenvectors @ (val * xy).T + center),color='#2ca02c')
        # plt.plot(*(2.44765*eigenvectors @ (val * xy).T + center),label='95% confidence ellipse',color='#2ca02c')
        if legend:
            plt.legend(loc='upper right')
        plt.ticklabel_format(useOffset=False)
        plt.gca().locator_params(nbins=5)
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
    else:
        pass
    return covariance,eigenvectors,val,eigenvalues,center

def point_intersection(scatter1,scatter2,vizualize=False): # Calculates number of points intersecting with other scatter's ellipse and vice versa
    # plt.scatter(scatter1[3,:],scatter1[2,:])
    # # plt.scatter(scatter2[3,:],scatter2[2,:])
    ## Point intersection
    covariance1 = np.cov(scatter1[3,:].astype('float'),scatter1[2,:].astype('float'))
    covariance2 = np.cov(scatter2[3,:].astype('float'),scatter2[2,:].astype('float'))
    
    eigenvalues1, eigenvectors1 = np.linalg.eig(covariance1)
    eigenvalues2, eigenvectors2 = np.linalg.eig(covariance2)
    val1 = np.sqrt(eigenvalues1)
    val2 = np.sqrt(eigenvalues2)
    center1 = np.mean([scatter1[3,:].astype('float'),scatter1[2,:].astype('float')],axis=1)[:,None]
    center2 = np.mean([scatter2[3,:].astype('float'),scatter2[2,:].astype('float')],axis=1)[:,None]
    intersection_points = 0
    for point_idx in range(scatter1.shape[1]):
        point1 = scatter1[2:4,point_idx]
        point1 = np.flip(point1)
        point2 = scatter2[2:4,point_idx]
        point2 = np.flip(point2)
        # plt.scatter(point[1],point[0])
        if (point1 - center2.flatten()) @ (np.linalg.inv((5.991)*covariance2)) @ (point1 - center2.flatten()) < 1:
            intersection_points += 1 #skaiciuojam ar point in ar out ellipse 
            plt.scatter(point1[0],point1[1],color='red')
        if (point2 - center1.flatten()) @ (np.linalg.inv((5.991)*covariance1)) @ (point2 - center1.flatten()) < 1:
            intersection_points += 1 #skaiciuojam ar point in ar out ellipse
            plt.scatter(point2[0],point2[1],color='red')
    if intersection_points > 0 and vizualize == True:
        vizualize_ellipse(scatter1[2:4,:],True,False,False)
        vizualize_ellipse(scatter2[2:4,:],True,False,False)
    return intersection_points

def ellipsoidal_area_intersection(scatter1,scatter2): # calculates intersection area of two ellipses that bounds two scatters
    covariance1 = np.cov(scatter1[3,:].astype('float'),scatter1[2,:].astype('float'))
    covariance2 = np.cov(scatter2[3,:].astype('float'),scatter2[2,:].astype('float'))
    eigenvalues1, eigenvectors1 = np.linalg.eig(covariance1)
    eigenvalues2, eigenvectors2 = np.linalg.eig(covariance2)
    val1 = np.sqrt(eigenvalues1)
    val2 = np.sqrt(eigenvalues2)
    center1 = np.mean([scatter1[3,:].astype('float'),scatter1[2,:].astype('float')],axis=1)[:,None]
    center2 = np.mean([scatter2[3,:].astype('float'),scatter2[2,:].astype('float')],axis=1)[:,None]
    ellipse1 = 2.44765*eigenvectors1 @ (val1 * xy).T + center1
    ellipse2 = 2.44765*eigenvectors2 @ (val2 * xy).T + center2
    polygon1 = Polygon(ellipse1.T)
    polygon2 = Polygon(ellipse2.T)
    intersection = polygon1.intersection(polygon2)
    if not intersection.is_empty:
        
        intersection_area = intersection.area
        # print(f"The area of intersection is: {intersection_area}")
        # print(f"Intersection percentage {round((intersection_area*100)/(polygon1.area + polygon2.area - intersection_area),2)}%")
        iou = (intersection_area*100)/(polygon1.area + polygon2.area - intersection_area)
        intersection_x,intersection_y = intersection.exterior.xy
        plt.fill(intersection_x,intersection_y,alpha=0.4,color='y')
        
        # inset_position = [0.65, 0.65, 0.25, 0.25]  # [left, bottom, width, height] in figure coordinates
        # inset_ax = plt.axes(inset_position)  # Create inset axes
        # inset_ax.ticklabel_format(useOffset=False)
        # inset_ax.fill(intersection_x,intersection_y,alpha=0.4,color='y')
        # inset_ax.set_xlim(12.2139, 12.2142)
        # inset_ax.set_ylim(54.5513, 54.5514)
        # # inset_ax.set_xticklabels([])
        # inset_ax.set_title("Zoomed in intersection", fontsize=10)
        # inset_ax.tick_params(labelsize=8)  # Smaller ticks for inset

        return intersection.area,iou
    else:
        return 0,0

def z_score_bounding_box(vessel_cluster, z_score):
    vessel1_long_m,vessel1_lat_m = np.mean([vessel_cluster[3,:],vessel_cluster[2,:]],axis=1)
    vessel1_long_std,vessel1_lat_std = np.std(np.array(vessel_cluster[3,:])),np.std(np.array(vessel_cluster[2,:]))
    
    vessel1_long_lim_min = vessel1_long_m  - vessel1_long_std*z_score
    vessel1_long_lim_max = vessel1_long_m  + vessel1_long_std*z_score
    vessel1_lat_lim_min = vessel1_lat_m  - vessel1_lat_std*z_score
    vessel1_lat_lim_max = vessel1_lat_m  + vessel1_lat_std*z_score
    coord1 = [vessel1_long_lim_min,vessel1_lat_lim_min]
    coord2 = [vessel1_long_lim_min,vessel1_lat_lim_max]
    coord3 = [vessel1_long_lim_max,vessel1_lat_lim_max]
    coord4 = [vessel1_long_lim_max,vessel1_lat_lim_min]
    square_coord = np.vstack((coord1,coord2,coord3,coord4,coord1)).T
    return square_coord

def square_intersection_area(square1,square2):
    sq1_b_l =np.min(square1,axis=1)
    sq1_t_r= np.max(square1,axis=1)
    sq2_b_l =np.min(square2,axis=1)
    sq2_t_r= np.max(square2,axis=1)
    x_l = max(sq1_b_l[0],sq2_b_l[0])
    y_b = max(sq1_b_l[1],sq2_b_l[1])
    x_r = min(sq1_t_r[0],sq2_t_r[0])
    y_t = min(sq1_t_r[1],sq2_t_r[1])
    coord1 = [x_l,y_b]
    coord2 = [x_l,y_t]
    coord3 = [x_r,y_t]
    coord4 = [x_r,y_b]
    if x_l < x_r and y_b < y_t:
        width = x_r - x_l
        height = y_t - y_b
        area = width * height
        sq1_area = (sq1_t_r[0] - sq1_b_l[0]) * (sq1_t_r[1] - sq1_b_l[1])
        sq2_area = (sq2_t_r[0] - sq2_b_l[0]) * (sq2_t_r[1] - sq2_b_l[1])
        iou = area/(sq1_area+sq2_area-area)
        intersect_coords = np.vstack((coord1,coord2,coord3,coord4,coord1)).T
        return area,iou,intersect_coords
    else:
        # print('Squares do not intersect')
        return 0,0,[]
    
def gmm(scatter1,scatter2,plot_gmm=True): #Creates GMM framework to vizualize and calculate GMM intersection area
    np.random.seed(1)
    cluster_1 = scatter1[2:4].T  # Cluster 1 (centered at [-2, -2])
    cluster_2 = scatter2[2:4].T    # Cluster 2 (centered at [2, 2])

    # # Combine the clusters into one array
    data = np.vstack([cluster_1, cluster_2])
    df = pd.DataFrame(data, columns=['Latitude', 'Longitude'])
    gmm = GaussianMixture(n_components=2, covariance_type='full', random_state=42)
    gmm.fit(data)
    labels = gmm.predict(data)  # Cluster assignments
    labels = np.array([0] * 10 + [1] * 10)
    df['Cluster'] = labels
    if plot_gmm:
        g = sns.JointGrid(data=df, x='Longitude', y='Latitude')
        g.ax_joint.ticklabel_format(useOffset=False)

        # Add scatter points for each cluster with different colors
        colors = ['red', 'blue']
        for cluster_id, color in enumerate(colors):
            cluster_data = df[df['Cluster'] == cluster_id]
            g.ax_joint.scatter(
                cluster_data['Longitude'],
                cluster_data['Latitude'],
                label=f'Cluster {cluster_id + 1}',
                color=color,
                s=20,
                alpha=1
        )

    kde1_long = gaussian_kde(np.array(cluster_1[:,1],dtype='float'))
    kde2_long = gaussian_kde(np.array(cluster_2[:,1],dtype='float'))
    kde1_lat = gaussian_kde(np.array(cluster_1[:,0],dtype='float'))
    kde2_lat = gaussian_kde(np.array(cluster_2[:,0],dtype='float'))

    x_long = np.linspace(min(min(cluster_1[:,1]), min(cluster_2[:,1]))-0.001, max(max(cluster_1[:,1]), max(cluster_2[:,1]))+0.001, 1000)
    x_lat = np.linspace(min(min(cluster_1[:,0]), min(cluster_2[:,0]))-0.001, max(max(cluster_1[:,0]), max(cluster_2[:,0]))+0.001, 1000)



    kde1_values_long = kde1_long(x_long)
    kde2_values_long = kde2_long(x_long)

    kde1_values_lat = kde1_lat(x_lat)
    kde2_values_lat = kde2_lat(x_lat)
    if plot_gmm:
        g.ax_marg_x.plot(x_long, kde1_values_long, label='Cluster Data 1', lw=2,color=colors[0])
        g.ax_marg_x.plot(x_long, kde2_values_long, label='Cluster Data 2', lw=2,color=colors[1])
        g.ax_marg_x.fill_between(x_long, kde1_values_long, alpha=0.2, label='Intersection',color='red')
        g.ax_marg_x.fill_between(x_long, kde2_values_long, alpha=0.2, label='Intersection',color='blue')
        g.ax_marg_x.fill_between(x_long, np.minimum(kde1_values_long, kde2_values_long), alpha=0.5, label='Intersection',color='yellow')

        g.ax_marg_y.plot(kde1_values_lat,x_lat, label='Cluster Data 1', lw=2,color=colors[0])
        g.ax_marg_y.plot(kde2_values_lat,x_lat, label='Cluster Data 2', lw=2,color=colors[1])
        g.ax_marg_y.fill_betweenx(x_lat,kde1_values_lat, alpha=0.2, label='Intersection',color='red')
        g.ax_marg_y.fill_betweenx(x_lat,kde2_values_lat, alpha=0.2, label='Intersection',color='blue')
        g.ax_marg_y.fill_betweenx(x_lat,np.minimum(kde1_values_lat, kde2_values_lat), alpha=0.5, label='Intersection',color='yellow')
    
    intersection_area_long = np.trapz(np.minimum(kde1_values_long, kde2_values_long), x_long)
    intersection_area_lat = np.trapz(np.minimum(kde1_values_lat, kde2_values_lat), x_long)
    if plot_gmm:
        print(f"Longitude intersection area {intersection_area_long}")
        print(f"Latitude intersection area {intersection_area_lat}")
        print(f"Intersection area multiplied {intersection_area_long*intersection_area_lat}")
        print('------')
        plt.show()
    return intersection_area_long*intersection_area_lat
def n_root(x,n):
    return x**(1./n)    
def n_logistic(x,n):
    return (1-(1./(1+math.e**(n*(x-0.5)))))

### ELLIPSOIDAL POINT INTERSECTION

In [ ]:


plt.ticklabel_format(useOffset=False) # Not using scientific notation
t = np.linspace(0,2.0 * np.pi,1000) # Helper datasets for building vizualiziations
xy = np.stack((np.cos(t),np.sin(t)),axis=-1)
for interaction in tqdm(interactions):
    plt.plot(X_test_new[interaction[0],:,3],X_test_new[interaction[0],:,2]) #first ship predictor

    plt.plot(X_test_new[interaction[1],:,3],X_test_new[interaction[1],:,2]) #second ship predictor
    
    intersection = np.intersect1d(y_test_new[interaction[0],:,1],y_test_new[interaction[1],:,1]) # only data where timepoints match on an interaction
    palette = sns.color_palette("viridis_r", len(intersection))
    collision_counter = 0
    for i,timepoint in enumerate(intersection):
        plt.ticklabel_format(useOffset=False)
        plt.gca().locator_params(nbins=5)

        scatter1 = new_predictions[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],:,:] 
        scatter2 = new_predictions[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]

        plt.scatter(scatter1[3,:],scatter1[2,:],color=palette[i],alpha=0.5)
        plt.scatter(scatter2[3,:],scatter2[2,:],color=palette[i],alpha=0.5)

        covariance1,eigenvectors1,val1,eigenvalues1,center1 = vizualize_ellipse(scatter1[2:4,:],False,False,False,False)
        covariance2,eigenvectors2,val2,eigenvalues2,center2 = vizualize_ellipse(scatter2[2:4,:],False,False,False,False)

        plt.plot(*(2.44765*eigenvectors1 @ (val1 * xy).T + center1),color=palette[i])
        plt.plot(*(2.44765*eigenvectors2 @ (val2 * xy).T + center2),color=palette[i])
        intersection_points = point_intersection(scatter1,scatter2,False)
        # print(intersection_points)
        

        plt.scatter(y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='red') # first vessel target variable
        plt.scatter(y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='orange') # second vessel target variable
        plt.show() # comment out if all prediction regions wanted to be seen together
    


### ELLIPSOIDAL AREA INTERSECTION

In [ ]:



plt.ticklabel_format(useOffset=False)
t = np.linspace(0,2.0 * np.pi,1000)
xy = np.stack((np.cos(t),np.sin(t)),axis=-1)
for interaction in tqdm(interactions):
    plt.plot(X_test_new[interaction[0],:,3],X_test_new[interaction[0],:,2]) #first ship predictor

    plt.plot(X_test_new[interaction[1],:,3],X_test_new[interaction[1],:,2]) #second ship predictor
    
    intersection = np.intersect1d(y_test_new[interaction[0],:,1],y_test_new[interaction[1],:,1])
    palette = sns.color_palette("viridis_r", len(intersection))
    for i,timepoint in enumerate(intersection):
        plt.ticklabel_format(useOffset=False)
        scatter1 = new_predictions[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        scatter2 = new_predictions[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        plt.scatter(scatter1[3,:],scatter1[2,:],color=palette[i],alpha=0.5)
        plt.scatter(scatter2[3,:],scatter2[2,:],color=palette[i],alpha=0.5)
        covariance1,eigenvectors1,val1,eigenvalues1,center1 = vizualize_ellipse(scatter1[2:4,:],False,False,False,False)
        covariance2,eigenvectors2,val2,eigenvalues2,center2 = vizualize_ellipse(scatter2[2:4,:],False,False,False,False)
        plt.plot(*(2.44765*eigenvectors1 @ (val1 * xy).T + center1),color=palette[i])
        plt.plot(*(2.44765*eigenvectors2 @ (val2 * xy).T + center2),color=palette[i])
        plt.scatter(y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='red')
        plt.scatter(y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='orange')
        area,iou = ellipsoidal_area_intersection(scatter1,scatter2)
        print(area,iou)
        plt.show()
        
        
    

### Z-SCORE INTERSECTION AREA

In [ ]:



for interaction in tqdm(interactions):

    intersection = np.intersect1d(y_test_new[interaction[0],:,1],y_test_new[interaction[1],:,1])
    palette = sns.color_palette("viridis_r", len(intersection))
    for i,timepoint in enumerate(intersection):
        plt.ticklabel_format(useOffset=False)
        scatter1 = new_predictions[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        scatter2 = new_predictions[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        plt.scatter(scatter1[3,:],scatter1[2,:],color=palette[i],alpha=0.5)
        plt.scatter(scatter2[3,:],scatter2[2,:],color=palette[i],alpha=0.5)
        square_coord1 = z_score_bounding_box(scatter1,3)
        square_coord2 = z_score_bounding_box(scatter2,3)
        area, iou,intersect_coords= square_intersection_area(square_coord1,square_coord2)
        plt.scatter(y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='red')
        plt.scatter(y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],3],y_test_new[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],2],color='orange')
        print(area,iou)
        plt.plot(square_coord1[0],square_coord1[1],color=palette[i])
        plt.plot(square_coord2[0],square_coord2[1],color=palette[i])
        if area > 0:
            plt.fill(intersect_coords[0],intersect_coords[1],color = 'y',alpha= 0.3)
        plt.show()        
        

### GMM

In [ ]:
gmm_interactions = []
for interaction in tqdm(interactions):

    intersection = np.intersect1d(y_test_new[interaction[0],:,1],y_test_new[interaction[1],:,1])
    palette = sns.color_palette("viridis_r", len(intersection))
    highest_per_interaction = 0
    for i,timepoint in enumerate(intersection):
        scatter1 = new_predictions[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        scatter2 = new_predictions[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        score = gmm(scatter1,scatter2)
        if score > highest_per_interaction:
            highest_per_interaction = score
    gmm_interactions.append([highest_per_interaction,interaction[2]]) ## dataset with all highest score of GMM per interaction + showing whether its collision
gmm_interactions = np.array(gmm_interactions)    
        

In [ ]:
plt.scatter(gmm_interactions[np.where(gmm_interactions[:,1] == 1),0],gmm_interactions[np.where(gmm_interactions[:,1] == 1),1])
plt.scatter(gmm_interactions[np.where(gmm_interactions[:,1] == 0),0],gmm_interactions[np.where(gmm_interactions[:,1] == 0),1])
plt.xlabel('GMM intersection score')
plt.ylabel('Collision')

### SILHOUETTE VALUE

In [ ]:
silhouette_interactions = []
for interaction in tqdm(interactions):

    intersection = np.intersect1d(y_test_new[interaction[0],:,1],y_test_new[interaction[1],:,1])
    palette = sns.color_palette("viridis_r", len(intersection))
    lowest_per_interaction = 1
    for i,timepoint in enumerate(intersection):
        scatter1 = new_predictions[interaction[0],np.where(np.all(new_predictions[interaction[0],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        scatter2 = new_predictions[interaction[1],np.where(np.all(new_predictions[interaction[1],:,1,:] == timepoint,axis=1)==True)[0][0],:,:]
        cluster1_points = scatter1[2:4,:].T
        cluster2_points = scatter2[2:4,:].T
        combined_data = np.vstack([cluster1_points, cluster2_points])
        labels = np.array([0] * cluster1_points.shape[0] + [1] * cluster2_points.shape[0])
        silhouette_avg = silhouette_score(combined_data, labels)
        if silhouette_avg < lowest_per_interaction:
            lowest_per_interaction = silhouette_avg
    silhouette_interactions.append([1-lowest_per_interaction,interaction[2]])

In [ ]:
root_silhouette_interactions = []
logistic_silhouette_interactions = []
for i in silhouette_interactions:
    root_silhouette_interactions.append([n_root(i[0],2),i[1]])
for i in silhouette_interactions:
    logistic_silhouette_interactions.append([n_logistic(i[0],10),i[1]])
root_silhouette_interactions = np.array(root_silhouette_interactions)
logistic_silhouette_interactions = np.array(logistic_silhouette_interactions)
sns.kdeplot(x=root_silhouette_interactions[:,0], fill=True, color='blue', bw_adjust=0.2)
plt.xlabel('Silhouette score square root transformation')
plt.show()
sns.kdeplot(x=logistic_silhouette_interactions[:,0], fill=True, color='blue', bw_adjust=0.2)
plt.xlabel('Silhouette score logistic (n=10) transformation')
plt.show()